<a href="https://colab.research.google.com/github/Omarsawan/Feature-construction-and-Categorical-features-tutorial/blob/master/Feature_Construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


In this tutorial , I am going to talk about categorical features and using original features of the data set to construct new features.

# Categorical features


A categorical feature takes only a limited number of values , it doesn't have continous values .A categorical variable can take on one of a limited, and usually fixed, number of possible values, assigning each individual or other unit of observation to a particular group or category. 
Consider a survey that asks how often you eat breakfast and provides four options: "Never", "Rarely", "Most days", or "Every day". In this case, the data is categorical, because responses fall into a fixed set of categories.
If people responded to a survey about which brand of car they owned, the responses would fall into categories like "Honda", "Toyota", and "Ford". In this case, the data is also categorical.

# Examples

First I will import the data set to show examples about categorical features

In [1]:
#import panda library to be able to load the dataset
import pandas as pd
#read the dataset
data=pd.read_csv('https://raw.githubusercontent.com/Omarsawan/Feature-construction-and-Categorical-features-tutorial/master/data/ks-projects-201801.csv',parse_dates=['deadline', 'launched'], encoding='latin-1')
#show the first 7 rows
data.head(7)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,52375.0,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.0,successful,16,US,1205.0,1205.0,1000.00


So we can consider that categorical features are features with data type equal to object or date since we know that these data types don't have continuous values , there may be another types but i will consider this now.

In [3]:
#make a data series with index equal to the column name and the value of the index is whether this column is of type object
indicesObjects=(data.dtypes=='object')
#make a data series with index equal to the column name and the value of the index is whether this column is of type datetime
indicesDate=(data.dtypes=='datetime64[ns]')

#make a data series with columns that have true only
objectColumns=indicesObjects[indicesObjects]
dateColumns=indicesDate[indicesDate]

#show only the index (which is the column name of categorical features)
objectsList=list(objectColumns.index)
datetimeList=list(dateColumns.index)

categoricalFeatures=(objectsList+datetimeList)

print('Columns with data type object: ',objectsList)
print('Columns with data type datetime: ',datetimeList)
print('Categorical features are',categoricalFeatures)
print('Count of Categorical features is',len(categoricalFeatures))

Columns with data type object:  ['name', 'category', 'main_category', 'currency', 'state', 'country']
Columns with data type datetime:  ['deadline', 'launched']
Categorical features are ['name', 'category', 'main_category', 'currency', 'state', 'country', 'deadline', 'launched']
Count of Categorical features is 8


So we can say that our dataset has 8 categorical features .

# Approaches to handle categorical features

You will get an error if you try to plug these variables into most machine learning models in Python without preprocessing them first.So there are many approaches that you can use to prepare your categorical data .

The first and easiest approach is to just remove this variables from the data set .This approach will only work well if the columns did not contain useful information.

In [4]:
dropCategorical=data.select_dtypes(exclude='object').select_dtypes(exclude='datetime64[ns]')
dropCategorical.head(7)

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,1000.0,0.0,0,0.0,0.0,1533.95
1,1000003930,30000.0,2421.0,15,100.0,2421.0,30000.00
2,1000004038,45000.0,220.0,3,220.0,220.0,45000.00
3,1000007540,5000.0,1.0,1,1.0,1.0,5000.00
4,1000011046,19500.0,1283.0,14,1283.0,1283.0,19500.00
5,1000014025,50000.0,52375.0,224,52375.0,52375.0,50000.00
6,1000023410,1000.0,1205.0,16,1205.0,1205.0,1000.00


The second approach is Label encoding ,it assigns each unique value to a different integer.
This approach assumes an ordering of the categories: 
"Never" (0) < "Rarely" (1) < "Most days" (2) < "Every day" (3).
This assumption make sense in some examples but other examples may not make sense , in our data set we can see that it can make sense for 'name' column if we want to consider the values in lexicographically order , also can make sense with 'launched' and 'deadline' columns which are of date type as dates can be ordered. Not all categorical variables have a clear ordering in the values, but we refer to those that do as ordinal variables. 

Scikit-learn has a LabelEncoder class that can be used to get label encodings. We loop over the categorical variables and apply the label encoder separately to each column.

In [5]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_data = data.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in categoricalFeatures:
    label_data[col] = label_encoder.fit_transform(label_data[col])
label_data.head(7)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,8656,106,12,5,1802,1000.0,6455,0.0,1,0,9,0.0,0.0,1533.95
1,1000003930,3484,91,6,13,2533,30000.0,9586,2421.0,1,15,22,100.0,2421.0,30000.00
2,1000004038,9500,91,6,13,869,45000.0,2132,220.0,1,3,22,220.0,220.0,45000.00
3,1000007540,8995,88,10,13,569,5000.0,1200,1.0,1,1,22,1.0,1.0,5000.00
4,1000011046,1957,54,6,13,1761,19500.0,6259,1283.0,0,14,22,1283.0,1283.0,19500.00
5,1000014025,5326,120,7,13,1976,50000.0,7365,52375.0,3,224,22,52375.0,52375.0,50000.00
6,1000023410,7622,57,7,13,1517,1000.0,5069,1205.0,3,16,22,1205.0,1205.0,1000.00


The third approach is to use One-Hot Encoding , it creates new columns indicating the presence (or absence) of each possible value in the original data. 
For example if we have a column color which has only 3 categories : (RED,YELLOW,BLUE) , it converts it into 3 columns and if some row has the value 'RED' , then its encoding will be (1,0,0) , and if another row has the value 'YELLOW' then its encoding will be (0,1,0).
So  the corresponding one-hot encoding contains one column for each possible value, and one row for each row in the original dataset .

In contrast to label encoding, one-hot encoding does not assume an ordering of the categories. Thus, you can expect this approach to work particularly well if there is no clear ordering in the categorical data (e.g., "Red" is neither more nor less than "Yellow"). We refer to categorical variables without an intrinsic ranking as nominal variables.

One-hot encoding generally does not perform well if the categorical variable takes on a large number of values (i.e., you generally won't use it for variables taking more than 15 different values).


In [6]:
#lets count the count of unique values of each column
uniqueCount ={}
for col in categoricalFeatures:
    curCol=data.filter([col]).iloc[:,0]
    uniqueCount[col]=len(curCol.unique())
print(uniqueCount)

{'name': 9993, 'category': 155, 'main_category': 15, 'currency': 14, 'state': 6, 'country': 23, 'deadline': 2624, 'launched': 9999}


So lets make one hot encoding only in columns state and currency to be able to visualize them.

In [8]:
from sklearn.preprocessing import OneHotEncoder

cols=['state','currency']

# Make copy to avoid changing original data 
OH_data = data.copy()

# Apply one-hot encoder to the columns we have choosen
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(OH_data[cols]))

# One-hot encoding removed index; put it back
OH_cols.index = OH_data.index

# Remove categorical columns (will be replaced with one-hot encoding)
num_X = OH_data.drop(cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X = pd.concat([num_X, OH_cols], axis=1)

OH_X.head(7)

,ID,name,category,main_category,deadline,goal,launched,pledged,backers,country,usd pledged,usd_pledged_real,usd_goal_real,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,0,GB,0.0,0.0,1533.95,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,15,US,100.0,2421.0,30000.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,3,US,220.0,220.0,45000.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,1,US,1.0,1.0,5000.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,14,US,1283.0,1283.0,19500.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,1000014025,Monarch Espresso Bar,Restaurants,Food,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,224,US,52375.0,52375.0,50000.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,2014-12-21,1000.0,2014-12-01 18:30:44,1205.0,16,US,1205.0,1205.0,1000.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


The world is filled with categorical data. You will be a much more effective data scientist if you know how to use this common data type!

# Feature construction

Creating new features from the raw data is one of the best ways to improve your model , it can be done with the encoding methods which I introduced two of them and also can be done through different methods.

Lets consider our column 'launched' which is of timedate type , so instead of encoding it using different encoding methods , we can simply replace it with 4 columns which are the hour , day , month , year .

In [10]:
# Make copy to avoid changing original data 
data_copy = data.copy()

#add the four columns
data_copy=data_copy.assign(hour=data_copy.launched.dt.hour,
                           day=data_copy.launched.dt.day,
                           month=data_copy.launched.dt.month,
                           year=data_copy.launched.dt.year)
#remove the launched column
data_copy=data_copy.drop(['launched'], axis=1)

data_copy.head(7)

,ID,name,category,main_category,currency,deadline,goal,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,0.0,failed,0,GB,0.0,0.0,1533.95,12,11,8,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2421.0,failed,15,US,100.0,2421.0,30000.00,4,2,9,2017
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,220.0,failed,3,US,220.0,220.0,45000.00,0,12,1,2013
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,1.0,failed,1,US,1.0,1.0,5000.00,3,17,3,2012
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,1283.0,canceled,14,US,1283.0,1283.0,19500.00,8,4,7,2015
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,52375.0,successful,224,US,52375.0,52375.0,50000.00,13,26,2,2016
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,1205.0,successful,16,US,1205.0,1205.0,1000.00,18,1,12,2014


One of the easiest ways to create new features is by combining categorical variables. For example, if one record has the country "CA" and category "Music", you can create a new value "CA_Music". This is a new categorical feature that can provide information about correlations between categorical variables. This type of feature is typically called an interaction.
In general, you would build interaction features from all pairs of categorical features. You can make interactions from three or more features as well, but you'll tend to get diminishing returns.

Pandas lets us simply add string columns together like normal Python strings.

In [11]:
# Make copy to avoid changing original data 
data_copy = data.copy()
#make new feature
interactions = data['category'] + "_" + data['country']
#give this new column a name
interactions.name='category-country'
#add the column to the data
dataInteraction=pd.concat([interactions, data_copy], axis=1)

#remove the category and country columns
dataInteraction=dataInteraction.drop(['category','country'], axis=1)

dataInteraction.head(7)

,category-country,ID,name,main_category,currency,deadline,goal,launched,pledged,state,backers,usd pledged,usd_pledged_real,usd_goal_real
0,Poetry_GB,1000002330,The Songs of Adelaide & Abullah,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,0.0,0.0,1533.95
1,Narrative Film_US,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,100.0,2421.0,30000.00
2,Narrative Film_US,1000004038,Where is Hank?,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,220.0,220.0,45000.00
3,Music_US,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,1.0,1.0,5000.00
4,Film & Video_US,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,1283.0,1283.0,19500.00
5,Restaurants_US,1000014025,Monarch Espresso Bar,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,successful,224,52375.0,52375.0,50000.00
6,Food_US,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.0,successful,16,1205.0,1205.0,1000.00


Also we can make a new feature only by observing our dataset and its initial features , so our dataset is a collections of some information about projects that are launched at some date so we can see that we can make a new feature which is the count of the number of projects launched in the last week , maybe this count can affect our model and our prediction , since we have the launched date of each project , we can count the number of projects launched in the last week.

In [16]:
# First, create a Series with a timestamp index and the values in the series are the original index of rows
# then sort it by the timestamp index
launched = pd.Series(data.index, index=data.launched, name="count_last_week").sort_index()
launched.head(20)

launched
1970-01-01 01:00:00    2842
2009-06-03 22:47:17    3530
2009-06-08 09:22:38    4073
2009-06-20 22:15:06     267
2009-07-09 02:32:29    7120
2009-07-15 21:14:49    4803
2009-08-03 21:23:16    4224
2009-08-06 23:19:55     357
2009-08-15 19:52:42    7776
2009-08-18 04:36:55    4182
2009-08-19 01:15:12    6094
2009-08-20 01:51:02    2763
2009-08-23 19:02:36    6055
2009-08-24 18:37:01    8491
2009-08-28 04:33:33    7124
2009-08-30 23:45:47    3541
2009-09-03 06:52:15    2329
2009-09-13 22:14:48    5935
2009-09-14 02:27:27    9746
2009-09-14 23:41:52    9578
Name: count_last_week, dtype: int64

Using a time series as the index allows us to define the rolling window size in terms of hours, days, weeks , you can use .rolling() to select time periods as the window. For example launched.rolling('7d') creates a rolling window that contains all the data in the previous 7 days. The window contains the current record, so if we want to count all the previous projects but not the current one, we'll need to subtract 1.

In [17]:
count_last_week = launched.rolling('7d').count() - 1
count_last_week.head(20)

launched
1970-01-01 01:00:00    0.0
2009-06-03 22:47:17    0.0
2009-06-08 09:22:38    1.0
2009-06-20 22:15:06    0.0
2009-07-09 02:32:29    0.0
2009-07-15 21:14:49    1.0
2009-08-03 21:23:16    0.0
2009-08-06 23:19:55    1.0
2009-08-15 19:52:42    0.0
2009-08-18 04:36:55    1.0
2009-08-19 01:15:12    2.0
2009-08-20 01:51:02    3.0
2009-08-23 19:02:36    3.0
2009-08-24 18:37:01    4.0
2009-08-28 04:33:33    2.0
2009-08-30 23:45:47    2.0
2009-09-03 06:52:15    2.0
2009-09-13 22:14:48    0.0
2009-09-14 02:27:27    1.0
2009-09-14 23:41:52    2.0
Name: count_last_week, dtype: float64

In [18]:
#now that we have the counts, we need to adjust the index so we can join it with the other training data.
count_last_week.index = launched.values
count_last_week = count_last_week.reindex(data.index)
count_last_week.head(20)

0     39.0
1     20.0
2     19.0
3     14.0
4     45.0
5     42.0
6     29.0
7     37.0
8     32.0
9     77.0
10    28.0
11    21.0
12    53.0
13    36.0
14    40.0
15    49.0
16    44.0
17    48.0
18    19.0
19    28.0
Name: count_last_week, dtype: float64

In [19]:
#now join the new feature with the other data again using .join since we've matched the index.
data.join(count_last_week).head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,count_last_week
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95,39.0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00,20.0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00,19.0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00,14.0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00,45.0
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00,42.0
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00,29.0
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00,37.0
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00,32.0
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00,77.0


Now we constructed a new feature .